# **Preprocessing and Feature Engineering**

### **Imports**
---

In [431]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn import metrics

### **Load the data**
-----

In [432]:
train = pd.read_csv('../data/train_cleaned.csv')
train.head()

,Id,parcel_id,dwelling_type,zoning_type,lot_frontage,lot_area,street_surface,alley_surface,property_shape,property_flatness,...,screen_porch_size,pool_size,pool_quality,fence_quality,misc_features,misc_features_value,month_sold,year_sold,sale_type,SalePrice
0,544,531379050,60,RL,43.0,11492.0,Pave,NaN,IR1,Lvl,...,0.0,0.0,NaN,NaN,NaN,0.0,4,2009,WD,220000.0
1,153,535304180,20,RL,68.0,7922.0,Pave,NaN,Reg,Lvl,...,0.0,0.0,NaN,NaN,NaN,0.0,1,2010,WD,109000.0
2,318,916386060,60,RL,73.0,9802.0,Pave,NaN,Reg,Lvl,...,0.0,0.0,NaN,NaN,NaN,0.0,4,2010,WD,174000.0
3,255,906425045,50,RL,82.0,14235.0,Pave,NaN,IR1,Lvl,...,0.0,0.0,NaN,NaN,NaN,0.0,3,2010,WD,138500.0
4,2827,908186070,180,RM,35.0,3675.0,Pave,NaN,Reg,Lvl,...,0.0,0.0,NaN,NaN,NaN,0.0,6,2006,New,140000.0


In [433]:
test = pd.read_csv('../data/test_cleaned.csv')
test.head()

,Id,parcel_id,dwelling_type,zoning_type,lot_frontage,lot_area,street_surface,alley_surface,property_shape,property_flatness,...,three_season_porch_size,screen_porch_size,pool_size,pool_quality,fence_quality,misc_features,misc_features_value,month_sold,year_sold,sale_type
0,2658,902301120,190,RM,69.0,9142.0,Pave,Grvl,Reg,Lvl,...,0.0,0.0,0.0,NaN,NaN,NaN,0.0,4,2006,WD
1,2718,905108090,90,RL,NaN,9662.0,Pave,NaN,IR1,Lvl,...,0.0,0.0,0.0,NaN,NaN,NaN,0.0,8,2006,WD
2,2414,528218130,60,RL,58.0,17104.0,Pave,NaN,IR1,Lvl,...,0.0,0.0,0.0,NaN,NaN,NaN,0.0,9,2006,New
3,1989,902207150,30,RM,60.0,8520.0,Pave,NaN,Reg,Lvl,...,0.0,0.0,0.0,NaN,NaN,NaN,0.0,7,2007,WD
4,625,535105100,20,RL,NaN,9500.0,Pave,NaN,IR1,Lvl,...,0.0,185.0,0.0,NaN,NaN,NaN,0.0,7,2009,WD


----
### *Redo the data type updates which aren't reflected when reading in file*

In [434]:
train[['Id','parcel_id','dwelling_type','alley_surface','masonry_veneer_type','basement_height','basement_condition','basement_finished_rating', 'basement_walkouts','basement_finished_rating2', 'pool_quality','garage_quality',
       'garage_condition', 'fence_quality', 'misc_features', 'fireplace_quality', 'garage_finished','Garage Type'
      ]] = train[['Id','parcel_id','dwelling_type','alley_surface','masonry_veneer_type','basement_height','basement_condition','basement_finished_rating', 'basement_walkouts',
       'basement_finished_rating2', 'pool_quality','garage_quality', 'garage_condition', 'fence_quality', 'misc_features','fireplace_quality', 'garage_finished','garage_location']].applymap(lambda itm: 'None' if itm == 'NaN' else str(itm))

test[['Id','parcel_id','dwelling_type','alley_surface','masonry_veneer_type','basement_height','basement_condition','basement_finished_rating', 'basement_walkouts',
       'basement_finished_rating2', 'pool_quality','garage_quality', 'garage_condition', 'fence_quality', 'misc_features', 'fireplace_quality', 'garage_finished',
       'Garage Type']] = test[['Id','parcel_id','dwelling_type','alley_surface','masonry_veneer_type','basement_height','basement_condition','basement_finished_rating', 'basement_walkouts',
       'basement_finished_rating2', 'pool_quality','garage_quality', 'garage_condition', 'fence_quality', 'misc_features','fireplace_quality', 'garage_finished','garage_location']].applymap(lambda itm: 'None' if itm == 'NaN' else str(itm))

In [435]:
#to integers
train[['full_bathrooms_basement','half_bathrooms_basement',
      'garage_car_capacity']] = train[['full_bathrooms_basement','half_bathrooms_basement','garage_car_capacity']].astype(int)

test[['full_bathrooms_basement','half_bathrooms_basement',
      'garage_car_capacity']] = test[['full_bathrooms_basement','half_bathrooms_basement','garage_car_capacity']].astype(int)

----
## **Model Attempt 3:** *Running model with all numerical features* (pre-data type update)

In [436]:
features = [col for col in test._get_numeric_data().columns if col != 'SalePrice' and test[col].isnull().sum() == 0]

In [437]:
test[features]

,lot_area,house_quality,house_condition,year_built,year_remodeled,finished_area,finished_area2,unfinished_area,total_basement_area,first_fl_area,...,garage_size,wood_deck_size,open_porch_size,enclose_porch_size,three_season_porch_size,screen_porch_size,pool_size,misc_features_value,month_sold,year_sold
0,9142.0,6,8,1910,1950,0,0,1020,1020,908.0,...,440,0.0,60.0,112.0,0.0,0.0,0.0,0.0,4,2006
1,9662.0,5,4,1977,1977,0,0,1967,1967,1967.0,...,580,170.0,0.0,0.0,0.0,0.0,0.0,0.0,8,2006
2,17104.0,7,5,2006,2006,554,0,100,654,664.0,...,426,100.0,24.0,0.0,0.0,0.0,0.0,0.0,9,2006
3,8520.0,5,6,1923,2006,0,0,968,968,968.0,...,480,0.0,0.0,184.0,0.0,0.0,0.0,0.0,7,2007
4,9500.0,6,5,1963,1963,609,0,785,1394,1394.0,...,514,0.0,76.0,0.0,0.0,185.0,0.0,0.0,7,2009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873,8000.0,6,6,1974,1974,931,153,0,1084,1084.0,...,488,0.0,96.0,0.0,0.0,0.0,0.0,0.0,11,2007
874,14670.0,6,7,1966,1999,575,0,529,1104,1104.0,...,480,0.0,230.0,0.0,0.0,0.0,0.0,0.0,8,2008
875,8250.0,5,5,1968,1968,250,492,210,952,1211.0,...,322,0.0,63.0,0.0,0.0,0.0,0.0,0.0,8,2008
876,9000.0,4,6,1971,1971,616,0,248,864,864.0,...,528,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,2007


In [438]:
X = train[features]
y = train['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

print(f'Training Shape: {X_train.shape, y_train.shape}')
print(f'Test Shape: {X_test.shape, y_test.shape}')

Training Shape: ((954, 32), (954,))
Test Shape: ((318, 32), (318,))


In [439]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [440]:
#Training Score
lr.score(X_train, y_train)

0.8798556232216684

In [441]:
#Test Score
lr.score(X_test, y_test)

0.8742177245601109

In [442]:
# Make predictions on test data
preds = lr.predict(test[features])

preds.shape

(878,)

In [443]:
# Cross Val Score to evaluate model
cross_val_score(lr, X_train, y_train).mean()

0.8732402260371721

Solid R-Squared results but could be misleading due to the number of features modeled.

In [444]:
#adjusted R-Squared Training data
r2 = lr.score(X_train, y_train)
1 - (((1-r2) * (len(y-1))) / (len(y) - len(features) -1))

0.8766556519273303

In [445]:
#adjusted R-Squared Test data
r2 = lr.score(X_test, y_test)
1 - (((1-r2) * (len(y-1))) / (len(y) - len(features) -1))

0.8708675913159493

In [446]:
#Root Mean Squared Error
predictions = lr.predict(X_train)

rmse = np.sqrt(metrics.mean_squared_error(y_train, predictions))
rmse

18114.97521151219

In [447]:
pd.DataFrame(data = lr.coef_, index=X.columns, columns=['Coefficient'])

,Coefficient
lot_area,1.535501
house_quality,12858.956542
house_condition,4499.735297
year_built,349.845708
year_remodeled,224.718098
finished_area,19.229469
finished_area2,5.537130
unfinished_area,1.462566
total_basement_area,26.229166
first_fl_area,12.831313


In [448]:
# add sale price column to test data set based on model
test['SalePrice'] = preds
test.head()

,Id,parcel_id,dwelling_type,zoning_type,lot_frontage,lot_area,street_surface,alley_surface,property_shape,property_flatness,...,pool_size,pool_quality,fence_quality,misc_features,misc_features_value,month_sold,year_sold,sale_type,Garage Type,SalePrice
0,2658,902301120,190,RM,69.0,9142.0,Pave,Grvl,Reg,Lvl,...,0.0,nan,nan,nan,0.0,4,2006,WD,Detchd,136811.977955
1,2718,905108090,90,RL,NaN,9662.0,Pave,nan,IR1,Lvl,...,0.0,nan,nan,nan,0.0,8,2006,WD,Attchd,149698.832112
2,2414,528218130,60,RL,58.0,17104.0,Pave,nan,IR1,Lvl,...,0.0,nan,nan,nan,0.0,9,2006,New,Attchd,212086.097183
3,1989,902207150,30,RM,60.0,8520.0,Pave,nan,Reg,Lvl,...,0.0,nan,nan,nan,0.0,7,2007,WD,Detchd,117029.180689
4,625,535105100,20,RL,NaN,9500.0,Pave,nan,IR1,Lvl,...,0.0,nan,nan,nan,0.0,7,2009,WD,Attchd,182285.275593


In [449]:
# Create Data Frame with only Id and SalePrice
all_numerical_submission = test[['Id', 'SalePrice']]
all_numerical_submission.shape

(878, 2)

In [450]:
# set index for new DF
all_numerical_submission.set_index('Id', inplace=True)
all_numerical_submission.head()

,SalePrice
Id,
2658,136811.977955
2718,149698.832112
2414,212086.097183
1989,117029.180689
625,182285.275593


In [451]:
# save submission
all_numerical_submission.to_csv('../data/all_numerical_submission.csv')

----
## **Model Attempt 4:** *Running model with all numerical features* (data types updated)

In [452]:
features = [col for col in test._get_numeric_data().columns if col != 'SalePrice' and test[col].isnull().sum() == 0]

In [453]:
X = train[features]
y = train['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

print(f'Training Shape: {X_train.shape, y_train.shape}')
print(f'Test Shape: {X_test.shape, y_test.shape}')

Training Shape: ((954, 32), (954,))
Test Shape: ((318, 32), (318,))


In [454]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [455]:
#Training Score
lr.score(X_train, y_train)

0.8798556232216684

In [456]:
#Test Score
lr.score(X_test, y_test)

0.8742177245601109

In [457]:
# Make predictions on test data
preds = lr.predict(test[features])

preds.shape

(878,)

In [458]:
# Cross Val Score to evaluate model
cross_val_score(lr, X_train, y_train).mean()

0.8732402260371721

write some commentary on scores

In [459]:
#adjusted R-Squared Training data
r2 = lr.score(X_train, y_train)
1 - (((1-r2) * (len(y-1))) / (len(y) - len(features) -1))

0.8766556519273303

In [460]:
#adjusted R-Squared Test data
r2 = lr.score(X_test, y_test)
1 - (((1-r2) * (len(y-1))) / (len(y) - len(features) -1))

0.8708675913159493

In [461]:
#Root Mean Squared Error
predictions = lr.predict(X_train)

rmse = np.sqrt(metrics.mean_squared_error(y_train, predictions))
rmse

18114.97521151219

Predicted home sale price off by about $18,114 on average relative to observed price. 

In [462]:
pd.DataFrame(data = lr.coef_, index=X.columns, columns=['Coefficient'])

,Coefficient
lot_area,1.535501
house_quality,12858.956542
house_condition,4499.735297
year_built,349.845708
year_remodeled,224.718098
finished_area,19.229469
finished_area2,5.537130
unfinished_area,1.462566
total_basement_area,26.229166
first_fl_area,12.831313


In [463]:
# add sale price column to test data set based on model
test['SalePrice'] = preds
test.head()

,Id,parcel_id,dwelling_type,zoning_type,lot_frontage,lot_area,street_surface,alley_surface,property_shape,property_flatness,...,pool_size,pool_quality,fence_quality,misc_features,misc_features_value,month_sold,year_sold,sale_type,Garage Type,SalePrice
0,2658,902301120,190,RM,69.0,9142.0,Pave,Grvl,Reg,Lvl,...,0.0,nan,nan,nan,0.0,4,2006,WD,Detchd,136811.977955
1,2718,905108090,90,RL,NaN,9662.0,Pave,nan,IR1,Lvl,...,0.0,nan,nan,nan,0.0,8,2006,WD,Attchd,149698.832112
2,2414,528218130,60,RL,58.0,17104.0,Pave,nan,IR1,Lvl,...,0.0,nan,nan,nan,0.0,9,2006,New,Attchd,212086.097183
3,1989,902207150,30,RM,60.0,8520.0,Pave,nan,Reg,Lvl,...,0.0,nan,nan,nan,0.0,7,2007,WD,Detchd,117029.180689
4,625,535105100,20,RL,NaN,9500.0,Pave,nan,IR1,Lvl,...,0.0,nan,nan,nan,0.0,7,2009,WD,Attchd,182285.275593


In [464]:
# Create Data Frame with only Id and SalePrice
all_numerical_submission_types_update = test[['Id', 'SalePrice']]
all_numerical_submission_types_update.shape

(878, 2)

In [465]:
# set index for new DF
all_numerical_submission_types_update.set_index('Id', inplace=True)
all_numerical_submission_types_update.head()

,SalePrice
Id,
2658,136811.977955
2718,149698.832112
2414,212086.097183
1989,117029.180689
625,182285.275593


In [466]:
# save submission
all_numerical_submission_types_update.to_csv('../data/all_numerical_submission_types_update.csv')

------
### *Handling Categorical Variables*

In [467]:
train.head()

,Id,parcel_id,dwelling_type,zoning_type,lot_frontage,lot_area,street_surface,alley_surface,property_shape,property_flatness,...,pool_size,pool_quality,fence_quality,misc_features,misc_features_value,month_sold,year_sold,sale_type,SalePrice,Garage Type
0,544,531379050,60,RL,43.0,11492.0,Pave,nan,IR1,Lvl,...,0.0,nan,nan,nan,0.0,4,2009,WD,220000.0,Attchd
1,153,535304180,20,RL,68.0,7922.0,Pave,nan,Reg,Lvl,...,0.0,nan,nan,nan,0.0,1,2010,WD,109000.0,Detchd
2,318,916386060,60,RL,73.0,9802.0,Pave,nan,Reg,Lvl,...,0.0,nan,nan,nan,0.0,4,2010,WD,174000.0,BuiltIn
3,255,906425045,50,RL,82.0,14235.0,Pave,nan,IR1,Lvl,...,0.0,nan,nan,nan,0.0,3,2010,WD,138500.0,Detchd
4,2827,908186070,180,RM,35.0,3675.0,Pave,nan,Reg,Lvl,...,0.0,nan,nan,nan,0.0,6,2006,New,140000.0,Basment


In [468]:
# changing Y/N and Paved/Gravel Columns to 1's and 0's
train['street_surface'] = train['street_surface'].map({'Pave': 1, 'Grvl': 0})
train['central_air'] = train['central_air'].map({'Y': 1, 'N':0})

test['street_surface'] = test['street_surface'].map({'Pave': 1, 'Grvl': 0})
test['central_air'] = test['central_air'].map({'Y': 1, 'N':0})

train.head()

,Id,parcel_id,dwelling_type,zoning_type,lot_frontage,lot_area,street_surface,alley_surface,property_shape,property_flatness,...,pool_size,pool_quality,fence_quality,misc_features,misc_features_value,month_sold,year_sold,sale_type,SalePrice,Garage Type
0,544,531379050,60,RL,43.0,11492.0,1,nan,IR1,Lvl,...,0.0,nan,nan,nan,0.0,4,2009,WD,220000.0,Attchd
1,153,535304180,20,RL,68.0,7922.0,1,nan,Reg,Lvl,...,0.0,nan,nan,nan,0.0,1,2010,WD,109000.0,Detchd
2,318,916386060,60,RL,73.0,9802.0,1,nan,Reg,Lvl,...,0.0,nan,nan,nan,0.0,4,2010,WD,174000.0,BuiltIn
3,255,906425045,50,RL,82.0,14235.0,1,nan,IR1,Lvl,...,0.0,nan,nan,nan,0.0,3,2010,WD,138500.0,Detchd
4,2827,908186070,180,RM,35.0,3675.0,1,nan,Reg,Lvl,...,0.0,nan,nan,nan,0.0,6,2006,New,140000.0,Basment


In [469]:
# candidates for #'s rather than dummy variables
train['exterior_quality'] = train['exterior_quality'].map({'Po':0,'Fa':1,'TA':2,'Gd':3,'Ex':4})
train['exterior_condition'] = train['exterior_condition'].map({'Po':0,'Fa':1,'TA':2,'Gd':3,'Ex':4})

train['basement_height'] = train['basement_height'].map({np.nan:0,'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5})
train['basement_condition'] = train['basement_condition'].map({np.nan:0,'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5})

train['heating_quality'] = train['heating_quality'].map({'Po':0,'Fa':1,'TA':2,'Gd':3,'Ex':4})

train['kitchen_quality'] = train['kitchen_quality'].map({'Po':0,'Fa':1,'TA':2,'Gd':3,'Ex':4})

train['fireplace_quality'] = train['fireplace_quality'].map({np.nan:0,'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5})

train['garage_quality'] = train['garage_quality'].map({np.nan:0,'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5})
train['garage_condition'] = train['garage_condition'].map({np.nan:0,'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5})

train['pool_quality'] = train['pool_quality'].map({np.nan:0,'Fa':1,'TA':2,'Gd':3,'Ex':4})

train['fence_quality'] = train['fence_quality'].map({np.nan:0,'MnWw':1,'GdWo':2,'MnPrv':3,'GdPrv':4})


test['exterior_quality'] = test['exterior_quality'].map({'Po':0,'Fa':1,'TA':2,'Gd':3,'Ex':4})
test['exterior_condition'] = test['exterior_condition'].map({'Po':0,'Fa':1,'TA':2,'Gd':3,'Ex':4})

test['basement_height'] = test['basement_height'].map({np.nan:0,'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5})
test['basement_condition'] = test['basement_condition'].map({np.nan:0,'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5})

test['heating_quality'] = test['heating_quality'].map({'Po':0,'Fa':1,'TA':2,'Gd':3,'Ex':4})

test['kitchen_quality'] = test['kitchen_quality'].map({'Po':0,'Fa':1,'TA':2,'Gd':3,'Ex':4})

test['fireplace_quality'] = test['fireplace_quality'].map({np.nan:0,'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5})

test['garage_quality'] = test['garage_quality'].map({np.nan:0,'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5})
test['garage_condition'] = test['garage_condition'].map({np.nan:0,'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5})

test['pool_quality'] = test['pool_quality'].map({np.nan:0,'Fa':1,'TA':2,'Gd':3,'Ex':4})

test['fence_quality'] = test['fence_quality'].map({np.nan:0,'MnWw':1,'GdWo':2,'MnPrv':3,'GdPrv':4})

train.head(10)

,Id,parcel_id,dwelling_type,zoning_type,lot_frontage,lot_area,street_surface,alley_surface,property_shape,property_flatness,...,pool_size,pool_quality,fence_quality,misc_features,misc_features_value,month_sold,year_sold,sale_type,SalePrice,Garage Type
0,544,531379050,60,RL,43.0,11492.0,1,nan,IR1,Lvl,...,0.0,NaN,NaN,nan,0.0,4,2009,WD,220000.0,Attchd
1,153,535304180,20,RL,68.0,7922.0,1,nan,Reg,Lvl,...,0.0,NaN,NaN,nan,0.0,1,2010,WD,109000.0,Detchd
2,318,916386060,60,RL,73.0,9802.0,1,nan,Reg,Lvl,...,0.0,NaN,NaN,nan,0.0,4,2010,WD,174000.0,BuiltIn
3,255,906425045,50,RL,82.0,14235.0,1,nan,IR1,Lvl,...,0.0,NaN,NaN,nan,0.0,3,2010,WD,138500.0,Detchd
4,2827,908186070,180,RM,35.0,3675.0,1,nan,Reg,Lvl,...,0.0,NaN,NaN,nan,0.0,6,2006,New,140000.0,Basment
5,1956,535426130,60,RL,70.0,11606.0,1,nan,IR1,HLS,...,0.0,NaN,NaN,nan,0.0,9,2007,WD,135000.0,Attchd
6,2752,906380150,20,RL,64.0,7488.0,1,nan,IR1,Lvl,...,0.0,NaN,NaN,nan,0.0,2,2006,WD,183600.0,Attchd
7,1480,907418040,20,RL,74.0,9627.0,1,nan,Reg,Lvl,...,0.0,NaN,NaN,nan,0.0,6,2008,WD,193000.0,Attchd
8,324,923202105,20,RL,93.0,10114.0,1,nan,Reg,Lvl,...,0.0,NaN,NaN,nan,0.0,3,2010,WD,173500.0,Attchd
9,1311,902301140,70,RM,34.0,4571.0,1,Grvl,Reg,Lvl,...,0.0,NaN,NaN,nan,0.0,5,2008,COD,98000.0,Detchd


------
## **Model 5:** *Numbers replacing categorical variables*

In [470]:
features = [col for col in test._get_numeric_data().columns if col != 'SalePrice' and test[col].isnull().sum() == 0]

In [471]:
X = train[features]
y = train['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

print(f'Training Shape: {X_train.shape, y_train.shape}')
print(f'Test Shape: {X_test.shape, y_test.shape}')

lr = LinearRegression()
lr.fit(X_train, y_train)

#Training Score
print(f'Training Score: {lr.score(X_train, y_train)}')
#Test Score
print(f'Test Score: {lr.score(X_test, y_test)}')

Training Shape: ((954, 38), (954,))
Test Shape: ((318, 38), (318,))
Training Score: 0.8898394072586665
Test Score: 0.8859250764243847


In [472]:
# Make predictions on test data
preds = lr.predict(test[features])

preds.shape

(878,)

In [473]:
# Cross Val Score to evaluate model
cross_val_score(lr, X_train, y_train).mean()

0.8830207643831214

In [474]:
#adjusted R-Squared Training data
r2 = lr.score(X_train, y_train)
print(f'Adjusted R-Squared for training: {1 - (((1-r2) * (len(y-1))) / (len(y) - len(features) -1))}')

#adjusted R-Squared Test data
r2 = lr.score(X_test, y_test)
print(f'Adjusted R-Squared for test: {1 - (((1-r2) * (len(y-1))) / (len(y) - len(features) -1))}')

#Root Mean Squared Error
predictions = lr.predict(X_train)

rmse = np.sqrt(metrics.mean_squared_error(y_train, predictions))
print(f'Root Mean Squared Error: {rmse}')

Adjusted R-Squared for training: 0.8863550089481134
Adjusted R-Squared for test: 0.8823168671628688
Root Mean Squared Error: 17345.99239468719


Predicted home sale price off by about $17,345 on average relative to observed price. 

In [475]:
pd.DataFrame(data = lr.coef_, index=X.columns, columns=['Coefficient'])

,Coefficient
lot_area,1.661653
street_surface,18363.998918
house_quality,10392.844077
house_condition,4258.914176
year_built,283.800354
year_remodeled,99.459297
exterior_quality,9358.762511
exterior_condition,160.868523
finished_area,18.364799
finished_area2,4.612454


In [476]:
# add sale price column to test data set based on model
test['SalePrice'] = preds

# Create Data Frame with only Id and SalePrice
categories_converted_numbers = test[['Id', 'SalePrice']]

# set index for new DF
categories_converted_numbers.set_index('Id', inplace=True)
categories_converted_numbers.head()

# save submission
categories_converted_numbers.to_csv('../data/categories_converted_numbers.csv')

-----

### *Binarize the remaining categorical columns*

In [477]:
train = pd.get_dummies(data = train, columns = ['dwelling_type','street_surface','alley_surface','property_shape',
                                                'property_flatness','lot_position','property_slope','ames_neighborhood',
                                                'prox_to_transport','prox_to_transport_2','bldg_type', 'floors','house_quality','house_condition',
                                                'roof_style', 'roof_material','ext_covering','ext_covering2','masonry_veneer_type','foundation_type',
                                                'basement_walkouts','basement_finished_rating','basement_finished_rating2', 'heating_type','central_air',
                                                'electrical_setup', 'full_bathrooms_basement', 'half_bathrooms_basement', 'full_bathrooms_above_ground',
                                                'half_bathrooms_above_ground','bedrooms_above_ground', 'kitchens_above_ground', 'home_functionality',
                                                'garage_location', 'garage_finished', 'garage_car_capacity','driveway_surface','misc_features', 'sale_type'], 
                                                drop_first = True)      

test = pd.get_dummies(data = test, columns = ['dwelling_type','street_surface','alley_surface','property_shape',
                                                'property_flatness','lot_position','property_slope','ames_neighborhood',
                                                'prox_to_transport','prox_to_transport_2','bldg_type', 'floors','house_quality','house_condition',
                                                'roof_style', 'roof_material','ext_covering','ext_covering2','masonry_veneer_type','foundation_type',
                                                'basement_walkouts','basement_finished_rating','basement_finished_rating2', 'heating_type','central_air',
                                                'electrical_setup', 'full_bathrooms_basement', 'half_bathrooms_basement', 'full_bathrooms_above_ground',
                                                'half_bathrooms_above_ground','bedrooms_above_ground', 'kitchens_above_ground', 'home_functionality',
                                                'garage_location', 'garage_finished', 'garage_car_capacity','driveway_surface','misc_features', 'sale_type'], 
                                                drop_first = True)      

------
## **Model 6**: Binarized all remaining categorical columms

In [478]:
features = [col for col in test._get_numeric_data().columns if col != 'SalePrice' and test[col].isnull().sum() == 0 and col not in ['zoning_type_I (all)',
            'util_avail_NoSewr', 'ames_neighborhood_BrDale', 'prox_to_transport_2_PosA', 'floors_2.5Fin', 'roof_material_Metal', 'roof_material_Roll',
            'roof_material_WdShngl', 'ext_covering_AsphShn', 'ext_covering_PreCast', 'ext_covering2_Other', 'ext_covering2_PreCast', 'masonry_veneer_type_CBlock',
            'masonry_veneer_type_nan', 'foundation_type_Slab', 'basement_finished_rating_nan', 'basement_finished_rating2_nan', 'heating_type_GasA',
            'full_bathrooms_above_ground_4', 'kitchens_above_ground_1', 'garage_finished_nan', 'garage_car_capacity_1', 'sale_type_VWD']]      

In [484]:
X = train[features]
y = train['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

print(f'Training Shape: {X_train.shape, y_train.shape}')
print(f'Test Shape: {X_test.shape, y_test.shape}')

lr = LinearRegression()
lr.fit(X_train, y_train)

#Training Score
print(f'Training R2 Score: {lr.score(X_train, y_train)}')
#Test Score
print(f'Test R2 Score: {lr.score(X_test, y_test)}')

Training Shape: ((954, 216), (954,))
Test Shape: ((318, 216), (318,))
Training R2 Score: 0.9438147909531384
Test R2 Score: 0.8958599577789359


In [485]:
#adjusted R-Squared Training data
r2 = lr.score(X_train, y_train)
print(f'Adjusted R-Squared for training: {1 - (((1-r2) * (len(y-1))) / (len(y) - len(features) -1))}')

#adjusted R-Squared Test data
r2 = lr.score(X_test, y_test)
print(f'Adjusted R-Squared for test: {1 - (((1-r2) * (len(y-1))) / (len(y) - len(features) -1))}')

#Root Mean Squared Error
predictions = lr.predict(X_train)

rmse = np.sqrt(metrics.mean_squared_error(y_train, predictions))
print(f'Root Mean Squared Error: {rmse}')

Adjusted R-Squared for training: 0.9322582124098502
Adjusted R-Squared for test: 0.8744396836917597
Root Mean Squared Error: 12387.880867104492


In [481]:
# Make predictions on test data
preds = lr.predict(test[features])

preds.shape

(878,)

In [482]:
# Cross Val Score to evaluate model
cross_val_score(lr, X_train, y_train).mean()

0.8842496675798799

In [394]:
# add sale price column to test data set based on model
test['SalePrice'] = preds

# Create Data Frame with only Id and SalePrice
all_features_binarized = test[['Id', 'SalePrice']]

# set index for new DF
all_features_binarized.set_index('Id', inplace=True)
all_features_binarized.head()

# save submission
all_features_binarized.to_csv('../data/all_features_binarized.csv')

In [483]:
train.corr()[['SalePrice']].sort_values(by = 'SalePrice', ascending = False)

,SalePrice
SalePrice,1.000000
exterior_quality,0.688407
year_built,0.667750
basement_height,0.663840
above_ground_living_area,0.649481
...,...
masonry_veneer_type_None,-0.401485
house_quality_5,-0.429538
garage_location_Detchd,-0.508495
garage_finished_Unf,-0.567855


------

### *Interaction Features*

In [425]:
train.corr()[['house_quality','SalePrice','house_condition', 
              'finished_area','finished_area2','unfinished_area','total_basement_area']].sort_values(by = 'SalePrice',ascending = False)

,house_quality,SalePrice,house_condition,finished_area,finished_area2,unfinished_area,total_basement_area
SalePrice,0.796371,1.000000,-0.173832,0.279768,-0.039258,0.187020,0.559873
house_quality,1.000000,0.796371,-0.168692,0.078173,-0.088906,0.296921,0.439408
year_built,0.604040,0.667750,-0.418112,0.195640,-0.029440,0.169534,0.441272
above_ground_living_area,0.514370,0.649481,-0.182684,-0.005928,-0.073298,0.242840,0.275718
garage_car_capacity,0.513411,0.616495,-0.268073,0.098467,-0.048631,0.239908,0.408230
garage_yr_built,0.551479,0.608534,-0.360775,0.125040,-0.044423,0.183220,0.367779
year_remodeled,0.556614,0.605059,0.004092,0.051697,-0.051356,0.166921,0.255617
full_bathrooms_above_ground,0.552262,0.601140,-0.295773,-0.026847,-0.099112,0.314786,0.332582
garage_size,0.457066,0.592864,-0.238029,0.155417,-0.017964,0.192626,0.428952
total_basement_area,0.439408,0.559873,-0.259477,0.345926,0.052202,0.429738,1.000000


In [415]:
features = [col for col in test._get_numeric_data().columns if col != 'SalePrice' and test[col].isnull().sum() == 0]

In [426]:
train['finished_space'] = train['finished_area'] * train['finished_area2']
test['finished_space'] = test['finished_area'] * test['finished_area2']

train['basement_space'] = train['unfinished_area'] * train['total_basement_area']
test['basement_space'] = test['unfinished_area'] * test['total_basement_area']

train['overall_house_rating'] = train['house_quality'] * train['house_condition']
test['overall_house_rating'] = test['house_quality'] * test['house_condition']

In [427]:
train.corr()[['SalePrice','finished_space','basement_space','overall_house_rating']].sort_values(by = 'SalePrice', ascending = False)

,SalePrice,finished_space,basement_space,overall_house_rating
SalePrice,1.000000,0.004733,0.315218,0.484486
house_quality,0.796371,-0.044475,0.368103,0.664562
year_built,0.667750,-0.010483,0.287689,0.129993
above_ground_living_area,0.649481,-0.030874,0.260828,0.281234
garage_car_capacity,0.616495,-0.031878,0.322197,0.195122
garage_yr_built,0.608534,-0.032271,0.271338,0.134167
year_remodeled,0.605059,-0.042600,0.219723,0.415703
full_bathrooms_above_ground,0.601140,-0.071193,0.349798,0.205012
garage_size,0.592864,0.003559,0.299367,0.168284
total_basement_area,0.559873,0.111160,0.641513,0.140266


#### *Lack of significant imporvement from adding the interaction features chosen above*

In [ ]:
# candidates for dropping collinear?
house & exterior & garage - - quality/condition
'basement_height', 'basement_condition'

# check with correlation/plot if 90 or greater, drop 1 of them that's less correlated with price
droppping one of the quality / condition ones

In [9]:
# at this point save a version to model with to try all features
# also save model after before each (combining, interaction, dropping) - run all features through after each to see if any improvement

------
### *Combining Features*

In [8]:
# combine bedrooms
# combine bathrooms
# combine kitchens (above and below ground)
# combine area of home (finished below and above)

### *Dropping Collinear Features*

In [ ]:
# for exploration and modelling
#1) run model where all features includes - X = df.drop(columns = 'SalesPrice')

# Three different way to create X:
#X = ads[['TV', 'radio', 'newspaper']]

#X = ads.drop(columns = ['sales']) # recommended with high dimensional datasets

model without feature engineering, interaction terms, dropping collinear features manually for all variables